# 3D Radiomic Feature Extraction
## Extract radiomics features from 3D medical imaging volumes

This notebook extracts radiomic features from 3D medical images:
- **Input:** Raw 3D NIfTI volumes from imagesTr/labelsTr
- **Features:** Shape, Texture (GLCM, GLRLM, GLSZM), Intensity-based radiomic features
- **Output:** Feature matrices saved as pickle and CSV
- **Multi-class:** Extracts features for each structure separately (classes 1-4)

In [ ]:
import nibabel as nib
import numpy as np
import pandas as pd
from pathlib import Path
import radiomics
from radiomics import featureextractor
import logging
import warnings
import pickle
from collections import defaultdict
from tqdm import tqdm
import json

warnings.filterwarnings('ignore')
logging.getLogger('radiomics').setLevel(logging.ERROR)

print(f"PyRadiomics version: {radiomics.__version__}")
print(f"Radiomics successfully imported!")

## 1. Configuration

In [ ]:
# Paths
BASE_DIR = Path('C:/FeatureEx')
IMAGES_DIR = BASE_DIR / 'imagesTr' / 'imagesTr'
LABELS_DIR = BASE_DIR / 'labelsTr' / 'labelsTr'
OUTPUT_DIR = BASE_DIR / 'radiomics_3d'
OUTPUT_DIR.mkdir(exist_ok=True)

# Settings
STRUCTURE_CLASSES = [1, 2, 3, 4]  # Classes 1-4 are structures (0 is background)
EXTRACT_ALL_CLASSES = True  # Extract combined mask (all non-background)

print(f"Input directories:")
print(f"  Images: {IMAGES_DIR}")
print(f"  Labels: {LABELS_DIR}")
print(f"Output directory: {OUTPUT_DIR}")
print(f"\nExtraction settings:")
print(f"  Structure classes: {STRUCTURE_CLASSES}")
print(f"  Extract all non-background: {EXTRACT_ALL_CLASSES}")

## 2. Initialize Radiomics Extractor

In [ ]:
# Create custom parameter file for 3D radiomics
params = {
    'binWidth': 25,  # 25 HU bin width for CT-like data
    'resampledPixelSpacing': None,  # Keep original spacing
    'interpolator': 'sitkBSpline',
    'label': 1,  # Will be updated per extraction
    'imageType': {
        'Original': {},
        'Wavelet': {'wavelet': 'coif1'},
    },
    'featureClass': {
        'shape': {},
        'firstorder': {},
        'glcm': {},
        'glrlm': {},
        'glszm': {},
        'ngtdm': {},
        'gldm': {},
    }
}

print(f"Radiomics parameters configured:")
print(f"  Bin width: {params['binWidth']}")
print(f"  Image types: {list(params['imageType'].keys())}")
print(f"  Feature classes: {list(params['featureClass'].keys())}")

## 3. Get File Pairs

In [ ]:
# Get matching image-label pairs
image_files = sorted([f for f in IMAGES_DIR.glob('*.nii*')])
label_files = sorted([f for f in LABELS_DIR.glob('*.nii*')])

# Create mapping
image_map = {f.stem: f for f in image_files}
label_map = {f.stem: f for f in label_files}

# Find matching pairs
matching_pairs = set(image_map.keys()) & set(label_map.keys())
file_pairs = [(image_map[name], label_map[name]) for name in sorted(matching_pairs)]

print(f"File pairs found: {len(file_pairs)}")
print(f"\nFirst 5 pairs:")
for img_path, lbl_path in file_pairs[:5]:
    print(f"  {img_path.name} <-> {lbl_path.name}")

## 4. Helper Functions for Multi-class Masking

In [ ]:
def create_class_mask(label_data, class_label, structure_classes):
    """
    Create binary mask for a specific class.
    
    Args:
        label_data: 3D label array
        class_label: Label value to extract (int or None for all structures)
        structure_classes: List of structure class labels
    
    Returns:
        Binary mask array
    """
    if class_label is None:
        # Create mask for all non-background classes
        mask = np.isin(label_data, structure_classes).astype(np.uint8)
    else:
        # Create mask for specific class
        mask = (label_data == class_label).astype(np.uint8)
    
    return mask

def validate_mask(mask_data, min_voxels=100):
    """
    Validate mask has sufficient voxels.
    
    Args:
        mask_data: Binary mask array
        min_voxels: Minimum required voxels
    
    Returns:
        Boolean indicating validity
    """
    voxel_count = np.count_nonzero(mask_data)
    return voxel_count >= min_voxels

print("Mask helper functions defined.")

## 5. Radiomics Extraction Function

In [ ]:
def extract_features_for_class(image_path, label_path, class_label, extractor, structure_classes):
    """
    Extract radiomics features for a specific class.
    
    Args:
        image_path: Path to image NIfTI
        label_path: Path to label NIfTI
        class_label: Class to extract (or None for all structures)
        extractor: RadiomicsFeatureExtractor instance
        structure_classes: List of structure class labels
    
    Returns:
        Dictionary of features or None if extraction failed
    """
    try:
        # Load data
        image_nib = nib.load(image_path)
        image_array = image_nib.get_fdata()
        
        label_nib = nib.load(label_path)
        label_array = label_nib.get_fdata()
        
        # Create binary mask for this class
        mask_array = create_class_mask(label_array, class_label, structure_classes)
        
        # Validate mask
        if not validate_mask(mask_array):
            return None
        
        # Save mask to temporary NIfTI
        mask_nib = nib.Nifti1Image(mask_array.astype(np.uint8), label_nib.affine)
        mask_path = Path('/tmp/temp_mask.nii.gz')
        nib.save(mask_nib, mask_path)
        
        # Extract features
        features = extractor.execute(str(image_path), str(mask_path))
        
        # Clean up temp file
        mask_path.unlink(missing_ok=True)
        
        return dict(features)
    
    except Exception as e:
        return None

print("Feature extraction function defined.")

## 6. Extract Features from All Images

In [ ]:
# Initialize extractor
extractor = featureextractor.RadiomicsFeatureExtractor()

# Initialize storage
all_features = defaultdict(list)  # {class_label: [features_dict, ...]}
feature_names = None
sample_ids = []  # Track which sample each feature comes from
extraction_log = []

print(f"Starting feature extraction from {len(file_pairs)} image-label pairs...\n")

# Extract features from each pair
for pair_idx, (img_path, lbl_path) in enumerate(tqdm(file_pairs), 1):
    sample_name = img_path.stem
    log_entry = {'sample': sample_name, 'status': 'success', 'classes_extracted': []}
    
    # Extract for all non-background classes
    for class_label in STRUCTURE_CLASSES:
        features_dict = extract_features_for_class(
            img_path, lbl_path, class_label, extractor, STRUCTURE_CLASSES
        )
        
        if features_dict:
            # Store features
            all_features[class_label].append(features_dict)
            log_entry['classes_extracted'].append(class_label)
            
            # Extract feature names from first successful extraction
            if feature_names is None:
                feature_names = list(features_dict.keys())
    
    # Extract combined mask (all structures together)
    if EXTRACT_ALL_CLASSES:
        features_dict = extract_features_for_class(
            img_path, lbl_path, None, extractor, STRUCTURE_CLASSES
        )
        if features_dict:
            all_features['all_structures'].append(features_dict)
            log_entry['classes_extracted'].append('all_structures')
    
    extraction_log.append(log_entry)
    sample_ids.append(sample_name)

print(f"\nFeature extraction complete!")
print(f"Total feature types: {len(feature_names) if feature_names else 0}")
print(f"\nFeatures extracted per class:")
for class_label in STRUCTURE_CLASSES + ['all_structures']:
    count = len(all_features[class_label])
    print(f"  Class {class_label}: {count} samples")

## 7. Convert to DataFrames

In [ ]:
# Create DataFrames for each class
feature_dfs = {}

for class_label in STRUCTURE_CLASSES + (['all_structures'] if EXTRACT_ALL_CLASSES else []):
    if all_features[class_label]:
        df = pd.DataFrame(all_features[class_label])
        feature_dfs[f'class_{class_label}'] = df
        
        print(f"\nClass {class_label}:")
        print(f"  Samples: {len(df)}")
        print(f"  Features: {len(df.columns)}")
        print(f"  Shape: {df.shape}")
        print(f"  Sample columns: {list(df.columns[:5])}...")

## 8. Combine and Normalize Features

In [ ]:
from sklearn.preprocessing import StandardScaler

# Create combined feature matrix (all classes concatenated)
combined_features_list = []
class_labels_list = []
sample_ids_combined = []

for class_idx, class_label in enumerate(STRUCTURE_CLASSES):
    df_key = f'class_{class_label}'
    if df_key in feature_dfs:
        df = feature_dfs[df_key]
        combined_features_list.append(df.values)
        class_labels_list.extend([class_label] * len(df))
        sample_ids_combined.extend(df.index.tolist())

# Combine and create DataFrame
if combined_features_list:
    combined_array = np.vstack(combined_features_list)
    combined_df = pd.DataFrame(
        combined_array,
        columns=feature_names if feature_names else [f'feature_{i}' for i in range(combined_array.shape[1])]
    )
    combined_df['class'] = class_labels_list
    
    print(f"Combined feature matrix:")
    print(f"  Shape: {combined_df.shape}")
    print(f"  Samples: {len(combined_df)}")
    print(f"  Features: {len(feature_names) if feature_names else 0}")
    
    # Show class distribution
    print(f"\nClass distribution:")
    for class_label in STRUCTURE_CLASSES:
        count = (combined_df['class'] == class_label).sum()
        print(f"  Class {class_label}: {count} samples")

## 9. Feature Statistics and Summary

In [ ]:
# Feature statistics
if feature_names:
    print(f"Feature statistics (combined data):")
    print(f"\nTop 10 features by variance:")
    
    feature_variance = combined_df[feature_names].var()
    top_features = feature_variance.nlargest(10)
    
    for idx, (feat_name, variance) in enumerate(top_features.items(), 1):
        print(f"  {idx}. {feat_name}: {variance:.4f}")
    
    print(f"\nFeature summary:")
    print(f"  Mean features per sample: {len(feature_names)}")
    print(f"  Total features: {len(feature_names) * len(combined_df)}")
    print(f"  Data types represented: {len(set([feat.split('_')[0] for feat in feature_names]))}")

## 10. Save Features to Files

In [ ]:
# Save individual class features as CSV
for class_label in STRUCTURE_CLASSES:
    df_key = f'class_{class_label}'
    if df_key in feature_dfs:
        output_path = OUTPUT_DIR / f'radiomics_3d_class_{class_label}.csv'
        feature_dfs[df_key].to_csv(output_path, index=True)
        print(f"Saved: {output_path.name}")

# Save combined features
if len(combined_features_list) > 0:
    combined_csv_path = OUTPUT_DIR / 'radiomics_3d_combined.csv'
    combined_df.to_csv(combined_csv_path, index=True)
    print(f"Saved: {combined_csv_path.name}")

# Save all-structures features
if EXTRACT_ALL_CLASSES and 'all_structures' in feature_dfs:
    all_struct_path = OUTPUT_DIR / 'radiomics_3d_all_structures.csv'
    feature_dfs['all_structures'].to_csv(all_struct_path, index=True)
    print(f"Saved: {all_struct_path.name}")

## 11. Save Features as Pickle (for sklearn compatibility)

In [ ]:
# Save as pickle for later use with sklearn
pickle_data = {
    'feature_dfs': feature_dfs,
    'combined_df': combined_df if len(combined_features_list) > 0 else None,
    'feature_names': feature_names,
    'structure_classes': STRUCTURE_CLASSES,
    'metadata': {
        'total_samples': len(file_pairs),
        'total_features': len(feature_names) if feature_names else 0,
        'classes': STRUCTURE_CLASSES,
        'extraction_log': extraction_log
    }
}

pickle_path = OUTPUT_DIR / 'radiomics_3d_features.pkl'
with open(pickle_path, 'wb') as f:
    pickle.dump(pickle_data, f)

print(f"\nSaved pickle: {pickle_path.name}")
print(f"Pickle contains:")
print(f"  - Feature DataFrames for each class")
print(f"  - Combined DataFrame")
print(f"  - Feature names")
print(f"  - Metadata and extraction log")

## 12. Create Configuration Summary

In [ ]:
config_summary = {
    'extraction_info': {
        'total_samples': len(file_pairs),
        'successful_extractions': sum(len(features) for features in all_features.values()),
        'extraction_log_path': str(OUTPUT_DIR / 'extraction_log.json')
    },
    'features': {
        'total_feature_types': len(feature_names) if feature_names else 0,
        'feature_names': feature_names[:10] if feature_names else [],
        'num_features_truncated': len(feature_names) - 10 if feature_names and len(feature_names) > 10 else 0
    },
    'classes': {
        'structure_classes': STRUCTURE_CLASSES,
        'num_structures': len(STRUCTURE_CLASSES),
        'include_all_structures': EXTRACT_ALL_CLASSES
    },
    'data_distribution': {
        f'class_{cls}': len(all_features[cls]) for cls in STRUCTURE_CLASSES
    },
    'output_files': {
        'pickle': 'radiomics_3d_features.pkl',
        'combined_csv': 'radiomics_3d_combined.csv',
        'individual_csvs': [f'radiomics_3d_class_{cls}.csv' for cls in STRUCTURE_CLASSES],
        'config_summary': 'radiomics_3d_config.json',
        'extraction_log': 'extraction_log.json'
    }
}

# Save config
config_path = OUTPUT_DIR / 'radiomics_3d_config.json'
with open(config_path, 'w') as f:
    json.dump(config_summary, f, indent=2)

print(f"Configuration summary:")
for key, val in config_summary.items():
    print(f"\n{key}:")
    if isinstance(val, dict):
        for k, v in val.items():
            print(f"  {k}: {v}")
    else:
        print(f"  {val}")

## 13. Save Extraction Log

In [ ]:
# Save detailed extraction log
log_path = OUTPUT_DIR / 'extraction_log.json'
with open(log_path, 'w') as f:
    json.dump(extraction_log, f, indent=2)

print(f"Extraction log saved to: {log_path.name}")

# Print summary
successful = sum(1 for entry in extraction_log if entry['classes_extracted'])
print(f"\nExtraction summary:")
print(f"  Total samples processed: {len(extraction_log)}")
print(f"  Successful: {successful}")
print(f"  Success rate: {(successful/len(extraction_log))*100:.1f}%")

In [ ]:
# Test loading the pickle file
with open(pickle_path, 'rb') as f:
    loaded_data = pickle.load(f)

print(f"Verification - Loaded pickle contents:")
print(f"  Keys: {list(loaded_data.keys())}")
print(f"  Feature DataFrames: {list(loaded_data['feature_dfs'].keys())}")
print(f"  Number of features: {len(loaded_data['feature_names'])}")
print(f"  Structure classes: {loaded_data['metadata']['structure_classes']}")

if loaded_data['combined_df'] is not None:
    print(f"\nCombined DataFrame shape: {loaded_data['combined_df'].shape}")
    print(f"Column names (first 5): {list(loaded_data['combined_df'].columns[:5])}")

In [ ]:
# Example: Using extracted features for classification
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

print("Example: Classification using radiomic features\n")

if loaded_data['combined_df'] is not None:
    combined = loaded_data['combined_df']
    
    # Separate features and labels
    X = combined[loaded_data['feature_names']].values
    y = combined['class'].values
    
    # Split data
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.3, random_state=42, stratify=y
    )
    
    # Scale features
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    # Train classifier
    clf = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)
    clf.fit(X_train_scaled, y_train)
    
    # Evaluate
    y_pred = clf.predict(X_test_scaled)
    accuracy = accuracy_score(y_test, y_pred)
    
    print(f"Classification Results:")
    print(f"  Training samples: {len(X_train)}")
    print(f"  Test samples: {len(X_test)}")
    print(f"  Accuracy: {accuracy:.4f}")
    print(f"\nPer-class accuracy:")
    print(classification_report(y_test, y_pred))
    
    print(f"\nTop 10 most important features:")
    importances = clf.feature_importances_
    feature_importance_df = pd.DataFrame({
        'feature': loaded_data['feature_names'],
        'importance': importances
    }).sort_values('importance', ascending=False)
    
    for idx, row in feature_importance_df.head(10).iterrows():
        print(f"  {row['feature']}: {row['importance']:.6f}")

In [ ]:
print("\n" + "="*70)
print("3D RADIOMIC FEATURE EXTRACTION - COMPLETE")
print("="*70)

print(f"\nOutput Directory: {OUTPUT_DIR}")
print(f"\nGenerated Files:")

output_files = list(OUTPUT_DIR.glob('*'))
for file in sorted(output_files):
    size_mb = file.stat().st_size / (1024*1024)
    print(f"  - {file.name} ({size_mb:.2f} MB)")

print(f"\nExtraction Summary:")
print(f"  Total samples: {len(file_pairs)}")
print(f"  Features per structure: {len(feature_names) if feature_names else 0}")
print(f"  Classes: {STRUCTURE_CLASSES}")
print(f"  All-structures combined: {EXTRACT_ALL_CLASSES}")

print(f"\nNext Steps:")
print(f"  1. Load features: pickle.load(open('{OUTPUT_DIR / 'radiomics_3d_features.pkl'}', 'rb'))")
print(f"  2. Access combined features: loaded_data['combined_df']")
print(f"  3. Use for classification: X = combined_df[feature_names], y = combined_df['class']")
print(f"  4. Fuse with CNN features for multi-modal analysis")